# BFS

In [26]:
graph = {'A' : ['B','C','G'],
         'B' : [],
         'C' : ['D','E','F'],
         'D' : [],
         'E' : [],
         'F' : [],
         'G' : ['F']
        }

In [27]:
visited = []
queued = []

def bfs(visited, graph, node):
    visited.append(node)
    queued.append(node)

    while queued:
        m = queued.pop(0)
        print(m, end=" -> ")
        for neighbor in graph[m]:
            if neighbor not in visited:
                visited.append(neighbor)
                queued.append(neighbor)

print("BFS : ")
bfs(visited, graph, 'A')

BFS : 
A -> B -> C -> G -> D -> E -> F -> 

# DFS

In [34]:
graph = {
    'A' : ['B','C','H'],
    'B' : ['E'],
    'C' : ['D'],
    'D' : ['E','I'],
    'E' : ['F'],
    'F' : ['G'],
    'H' : ['I'],
    'G' : [],
    'I' : ['J'],
    'J' : []
}

In [29]:
visited = set()
def dfs(visited, graph, node):
    if node not in visited:
        print(node, end = " -> ")
        visited.add(node)
        for n in graph[node]:
            dfs(visited, graph, n)

dfs(visited, graph, 'A')

A -> B -> E -> F -> G -> C -> D -> I -> J -> H -> 

# UCS

In [55]:
import heapq

def ucs(graph, start, goal):
    frontier = [(0, start)]
    came_from = {}
    cost_so_far = {start:0}

    while frontier:
        current_cost, current_node = heapq.heappop(frontier)

        if current_node == goal:
            break

        for neighbor, cost in graph[current_node]:
            new_cost = current_cost + cost
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                heapq.heappush(frontier, (new_cost, neighbor))
                came_from[neighbor] = current_node

    path = []
    node = goal
    while node != start:
        path.append(node)
        node = came_from[node]
    path.append(start)
    path.reverse()
    return path, cost_so_far[goal]

graph = {
    'B' : [('H', 700), ('C', 350), ('M', 1500)],
    'H' : [('M', 700), ('B', 700)],
    'C' : [('K', 1700), ('B', 350)],
    'M' : [('K', 1800), ('B', 1500), ('H', 700)],
    'K' : [('C', 1700), ('M', 1800)]
}

path, cost = ucs(graph, 'B', 'K')
print(path)
print(cost)

['B', 'C', 'K']
2050


# A*

In [76]:
from queue import PriorityQueue

def a_start(graph, start, goal, heuristic):
    open_set = PriorityQueue()
    open_set.put((0,start))
    came_from = {}

    g_score = {node:float('inf') for node in graph}
    g_score[start]=0

    f_score = {node:float('inf') for node in graph}
    f_score[start] = heuristic[start]

    while not open_set.empty():
        _, current = open_set.get()
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
                path.append(start)
                return path[::-1], f_score[goal]

        for neighbor, cost in graph[current].items():
            tent_g_score = g_score[current]+cost
            if tent_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tent_g_score
                f_score[neighbor] = g_score[neighbor]+heuristic[neighbor]
                open_set.put((f_score[neighbor], neighbor))
    return None, float('inf')

In [77]:
graph = {
    'A' : {'B':1, 'C':2},
    'B' : {'C':4, 'E':2},
    'C' : {'D':1, 'G':3},
    'D' : {},
    'E' : {},
    'F' : {},
    'G' : {}
}

heuristic = {'A':6, 'B':5, 'C':4, 'D':7, 'E':3, 'F':2, 'G':0}

start, goal = 'A', 'G'
path, cost = a_start(graph, start, goal, heuristic)
print(f"Path:{path}, Cost:{cost}")

Path:['A', 'G'], Cost:5


# GBFS

In [84]:
import heapq

class Node:
    def __init__(self, position, h_value, parent=None):
        self.position=position
        self.h_value=h_value
        self.parent = parent

    def __lt__(self, other):
        return self.h_value < other.h_value

def heuristic(a,b):
    return abs(a[0]-b[0])+abs(a[1]-b[1])

def gbfs(start, goal, grid):
    open_list = []
    closed_list = set()

    start_node = Node(start, heuristic(start, goal))
    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.position == goal:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        closed_list.add(current_node.position)

        for neighbor in get_neighbors(current_node.position, grid):
            if neighbor not in closed_list:
                neighbor_node = Node(neighbor, heuristic(neighbor, goal), current_node)
                heapq.heappush(open_list, neighbor_node)

    return None

def get_neighbors(position, grid):
    x, y = position
    neighbors = []
    directions = [(-1,0), (1,0), (0,-1), (0,1)]

    for dx, dy in directions:
        nx, ny = x + dx, y + dy

        if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny]!=1:
            neighbors.append((nx, ny))
    return neighbors

grid = [
    [0,0,0,0,0],
    [0,1,1,0,0],
    [0,1,0,0,0],
    [0,1,0,1,0],
    [0,0,0,0,0]
]

start = (0,0)
goal = (4,4)

path = gbfs(start, goal, grid)

if path:
    print("PAth found : ",path)
else:
    print("No path found")

PAth found :  [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]


# Hill 

In [88]:
def hill_climbing(graph, start, goal):
    current_node = start
    path = []  
    total_cost = 0
    visited = set()
    visited.add(current_node)

    while True:
        neighbors = graph.get(current_node, [])

        if current_node == goal:
            return path, total_cost

        if not neighbors:
            return path, float('inf')  

        best_neighbor = None
        best_cost = float('inf')
        best_edge_cost = 0

        for neighbor, cost in neighbors:
            if neighbor not in visited:
                if cost < best_cost:  
                    best_cost = cost
                    best_neighbor = neighbor
                    best_edge_cost = cost

        if best_neighbor is None:
            return path, float('inf') 

        current_node = best_neighbor
        path.append(current_node)
        total_cost += best_edge_cost
        visited.add(current_node)

graph = {
    'M': [('G', 500), ('MP', 700), ('R', 800)],
    'G': [('R', 600), ('M', 500)],
    'MP': [('M', 700)],
    'R': [('D', 400), ('M', 800), ('G', 600)],
    'D': [('UP', 200), ('R', 400)],
    'UP': [('D', 200), ('J&K', 600)],
    'J&K': [('L', 300), ('UP', 600)],
}

path, cost = hill_climbing(graph, 'M', 'L')
print(f"Path: {path}, Total_cost: {cost}")


Path: ['G', 'R', 'D', 'UP', 'J&K', 'L'], Total_cost: 2600


# Beam

In [1]:
def objective_function(x):
    return -(x**2)+10*x

def beam_search(start, objective_function, beam_width=3, iterations=10):
    open_list = [start]
    for _ in range(iterations):
        neighbors = [x+1 for x in open_list]+[x-1 for x in open_list]
        scored_neighbors = [(neighbor, objective_function(neighbor)) for neighbor in neighbors]
        open_list = [x[0] for x in sorted(scored_neighbors, key=lambda x:x[1], reverse=True)[:beam_width]]
    return open_list[0]

start = 0
best_position = beam_search(start, objective_function, beam_width=3)

print("Best postion found:", best_position)
print("Obj values:", objective_function(best_position))

Best postion found: 6
Obj values: 24


# FOL

In [21]:
from sympy.logic.boolalg import And, Or, Not
from sympy.abc import x, y

rainy = lambda weather: f'{weather} is rainy'

facts = {
    'Cloudy' : True,
    'Clear' : False,
    'Windy' : False
}

def rule(weather):
    if facts.get(weather, False):
        return rainy(weather)

def query(weather):
    if rule(weather):
        return f"It is {weather} so the weather will be rainy"
    else:
        return f"It is {weather} so the weather will not be rainy"

query("Clear")

weather = ['Cloudy', 'Clear', 'Windy']

for w in facts.keys():
    print(query(w))

It is Cloudy so the weather will be rainy
It is Clear so the weather will not be rainy
It is Windy so the weather will not be rainy
